0 is next, 1 is not next

In [1]:
!pip3 install transformers
!pip3 install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch


In [3]:
import pandas as pd
from tqdm import tqdm  # for our progress bar

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Correct predictions

In [5]:
correct_examples = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/zero_shot/TCL02_sentences_correct.csv", header = 0)

In [6]:
incorrect_examples = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/zero_shot/TCL02_sentences_incorr.csv", header = 0)

In [7]:
correct_tokenized = [tokenizer(x, y, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device) for x, y in zip(correct_examples['text'], correct_examples['second'])]

In [8]:
len(correct_tokenized)

9231

In [9]:
incorr_tokenized = [tokenizer(x, y, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device) for x, y in zip(incorrect_examples['text'], incorrect_examples['second'])]

In [10]:
len(incorr_tokenized)

9232

In [11]:
corr_label = torch.LongTensor([0]).to(device)
incorr_label = torch.LongTensor([1]).to(device)

In [12]:
def nsp(tokenized, label):
    length = len(tokenized)
    correct_count = 0
    loss_sum = 0
    for input in tokenized:
        outputs = model(**input, labels=label)
        if torch.argmax(outputs.logits) == label:
            #print("correct")
            correct_count += 1
        loss_sum += outputs.loss.item()
        #print(outputs.loss.item())
        #print(torch.argmax(outputs.logits).item())
    return correct_count/length, loss_sum/length

In [13]:
accuracy, avg_loss = nsp(correct_tokenized, corr_label)

In [14]:
print(accuracy, avg_loss)

1.0 5.323776570092472e-06


In [15]:
accuracy_incor, avg_loss_incor = nsp(incorr_tokenized, incorr_label)

In [16]:
print(accuracy_incor, avg_loss_incor)

0.0 12.033355762888283


Results


BZJ

1.0, 3.4950136268556838e-06

0.0, 12.457190679656955

GUL

1.0, 4.382941439299723e-06

0.0, 12.154078875830335

HPB

1.0, 4.208546394612041e-05

0.0015546843314857375, 11.949778980367189

JNT

1.0, 3.788576340179159e-06

0.0, 12.312673076058207

KJV

0.9981692719335304, 0.00974865394877733

0.02196564348070966, 10.362151318617123

NRSV

0.9839482200647249, 0.08612077890585282

0.1839244110794719, 7.667701944324847

PCM

0.9997848537005164, 0.0016147888899463638

0.00010756157900397978, 12.346538027794121

SCO1904

0.9996701846965699, 0.002141148393389533

0.001483924154987634, 11.81556357454688

SIPB

1.0, 3.013241433533054e-06

0.0, 12.652974187701126

USNT

1.0, 4.964416167671422e-06

0.0, 12.093108104345367

EAB
1.0, 3.960664925471179e-06

0.0, 12.368377923694139

HUNB

1.0, 4.260353477753521e-06

0.0, 12.238798976585247

SWZZB1921

1.0, 3.6263502025873798e-06

0.0, 12.389838169658422

TCL02

1.0 5.323776570092472e-06

0.0 12.033355762888283

Fine Tuning

In [5]:
mixed_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/train_test/TCL02_sentences_mixed_train.csv", header = 0)

In [6]:
len(mixed_train)

7007

In [7]:
print(mixed_train)

      Unnamed: 0 book  chapt version  \
0              0  MAT      1   TCL02   
1              1  MAT      1   TCL02   
2              2  MAT      1   TCL02   
3              3  MAT      1   TCL02   
4              4  MAT      1   TCL02   
...          ...  ...    ...     ...   
7002        9226  REV     22   TCL02   
7003        9227  REV     22   TCL02   
7004        9228  REV     22   TCL02   
7005        9229  REV     22   TCL02   
7006        9230  REV     22   TCL02   

                                                   text  \
0     ibrahim oglu davut oglu isa mesihin soy kaydi ...   
1                meryemden mesih diye taninan isa dogdu   
2     buna gore ibrahimden davuta kadar toplam on do...   
3     isa mesihin dogumu soyle oldu annesi meryem yu...   
4     ama birlikte olmalarindan once meryemin kutsal...   
...                                                 ...   
7002  ben isa kilise lerle ilgili bu tanikligi sizle...   
7003  davutun koku ve soyu benim parlak sabah y

In [8]:
mixed_train_tokenized = tokenizer(mixed_train['text'].tolist(), mixed_train['second'].tolist(), return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [9]:
len(mixed_train_tokenized.token_type_ids)

7007

In [10]:
mixed_train_tokenized['labels'] = torch.LongTensor([mixed_train['label'].tolist()]).T

In [11]:
mixed_train_tokenized.to(device)

{'input_ids': tensor([[  101, 13477, 13958,  ...,     0,     0,     0],
        [  101, 21442,  6672,  ...,     0,     0,     0],
        [  101, 21122,  2050,  ...,     0,     0,     0],
        ...,
        [  101, 20934,  8934,  ...,     0,     0,     0],
        [  101,  2014,  5035,  ...,     0,     0,     0],
        [  101, 21122,  8017,  ...,     0,     0,     0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([[1],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1

In [12]:
class BibleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [13]:
dataset = BibleDataset(mixed_train_tokenized)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [14]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/438 [00:00<?, ?it/s]<ipython-input-12-94e17abbf154>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████| 438/438 [02:40<00:00,  2.73it/s, loss=0.57]


Unmixed testing

In [16]:
corr_label = torch.LongTensor([0]).to(device)
incorr_label = torch.LongTensor([1]).to(device)

In [17]:
def nsp(tokenized, label):
    length = len(tokenized)
    correct_count = 0
    loss_sum = 0
    for input in tokenized:
        outputs = model(**input, labels=label)
        if torch.argmax(outputs.logits) == label:
            #print("correct")
            correct_count += 1
        loss_sum += outputs.loss.item()
        #print(outputs.loss.item())
        #print(torch.argmax(outputs.logits).item())
    return correct_count/length, loss_sum/length

In [18]:
correct_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/train_test/TCL02_sentences_correct_test.csv", header = 0)

In [19]:
correct_test_tokenized = [tokenizer(x, y, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device) for x, y in zip(correct_test['text'], correct_test['second'])]

In [20]:
len(correct_test_tokenized)

2224

In [21]:
incorr_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/train_test/TCL02_sentences_incorr_test.csv", header = 0)

In [22]:
incorr_test_tokenized = [tokenizer(x, y, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device) for x, y in zip(incorr_test['text'], incorr_test['second'])]

In [23]:
len(incorr_test_tokenized)

2224

In [24]:
accuracy_corr, avg_loss_corr = nsp(correct_test_tokenized, corr_label)

In [25]:
print(accuracy_corr, avg_loss_corr)

0.5116906474820144 0.6178419957310495


In [26]:
accuracy_incor, avg_loss_incor = nsp(incorr_test_tokenized, incorr_label)

In [27]:
print(accuracy_incor, avg_loss_incor)

0.810251798561151 0.6107205409864834


BZJ

0.6100478468899522, 0.56339682844047

0.7959671907040328, 0.5678392945777979


GUL

0.6439499304589708, 0.5952249919591834

0.8428372739916551, 0.43979918151700115

HPB

0.6942915097073312, 0.516876955710418

0.8180237612286294, 0.4626124977665543

JNT

0.6137440758293838, 0.5913816904966975

0.8513879485443466, 0.4754071372662675

KJV

0.7911136757068667 0.4241299550838929

0.8003462204270052 0.4697855664046338

NRSV

0.7869198312236287, 0.4296622140679765

0.7990506329113924, 0.4505719909147764

PCM

0.6886917960088692, 0.5279901197548886

0.7764966740576497, 0.5211996397329804

SCO1904

0.7771812080536913, 0.4395704471001305

0.7134228187919464 0.5813329156813205

1 Epoch: 

0.7912751677852349 0.40726676801717765

0.6932885906040268 0.6257057366367654

SIPB

0.5808777429467085, 0.6736263701222366

0.8943573667711598, 0.3911046249739429

USNT

0.5243288590604027 0.6559654127211848

0.7770326906957251 0.5773299033351537

EAB

0.5763517528223411 0.5749006762332857

0.7653000594177065 0.6029633780231947

HUNB

0.5497658079625293 0.5818024547085282

0.8097189695550351 0.6070099037131883

SWZZB1921

0.43394980184940557 0.7440231492709986

0.8996036988110965 0.4588819650374598

TCL02

0.5116906474820144 0.6178419957310495

0.810251798561151 0.6107205409864834




Mixed testing

In [ ]:
mixed_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp_ling_final_project/train_test/SCO1904_sentences_mixed_test.csv", header = 0)

In [ ]:
mixed_test_tokenized = tokenizer(mixed_test['text'].tolist(), mixed_test['second'].tolist(), return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
mixed_test_tokenized['labels'] = torch.LongTensor([mixed_test['label'].tolist()]).T

In [ ]:
mixed_test_tokenized.to(device)

{'input_ids': tensor([[  101,  3648,  2532,  ...,     0,     0,     0],
        [  101,  1998,  2339,  ...,     0,     0,     0],
        [  101,  2030,  8814,  ...,     0,     0,     0],
        ...,
        [  101,  1998,  1996,  ...,     0,     0,     0],
        [  101,  1998,  2331,  ...,     0,     0,     0],
        [  101,  1998, 18353,  ...,     0,     0,     0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([[0],
        [1],
        [0],
        ...,
        [1],
        [0],
        [1

In [ ]:
dataset = BibleDataset(mixed_test_tokenized)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
epochs = 1
correct = 0
loss_sum = 0
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        prediction = torch.argmax(outputs.logits)
        #print(prediction, labels)
        if prediction == labels:
            correct += 1
        loss_sum += outputs.loss.item()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')

  0%|          | 0/1490 [00:00<?, ?it/s]<ipython-input-13-94e17abbf154>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 1490/1490 [00:29<00:00, 50.67it/s]


In [ ]:
correct/len(mixed_test)

0.7523489932885906

In [ ]:
loss_sum/len(mixed_test)

0.4933053784626843